# Organize directories

The first step in training a ML model on sequences of images acquired from the ProtectCode app is to ensure the directory structure is set up for subsequent data loading. 

These commands assume the following: 

1) That the top directory for the training data and this notebook are both located in ~/src/YPB-AI/ on the EC2 instance. To get a zipped directory from S3 to that directory on your EC2 instance, you can run the following bash command: "aws s3 cp s3://your-bucket-name/yourZipFile.zip ~/src/YPB-AI/yourZipFile.zip". And to unzip it, you can run "unzip yourZipFile.zip"

2) That the jupyter notebook server was opened from the ~/src/ directory on the EC2 instance via the command "jupyter notebook --ip=0.0.0.0 --no-browser".

3) That underneath the top directory (which I usually call "Training" or something of the like), there are a series of subdirectories named and organized by classification label. For instance, each subdirectory that contains true negative image sequences, according to my implementation, contains the word "Negative". And subdirectories with true positives do not contain the word "Negative". 



In [2]:
import numpy as np
from glob import glob
import os, sys
import re

Set the name of your top data directory below. 

In [1]:
# topDirName = 'Training'
topDirName = 'Bam1_no_pillBottle'

Lump together all subdirectories with the same classification label. 

In [9]:
#Assumes current working directory is ~/src/YPB-AI
allSamples = np.array(glob(topDirName + '/*'))
os.system('mkdir ' + topDirName + '/Absent')
os.system('mkdir ' + topDirName + '/Present')

for sample in allSamples: 
    if 'Negative' in sample or 'Sat' in sample:
        cmdString = "mv " + sample + " " + topDirName + "/Absent"
    else:
        cmdString = "mv " + sample + " " + topDirName + "/Present"
    os.system(cmdString)

Each directory within the classification subdirectory contains a series of folders that have the following contents:

1) A .txt file containing some timing and other metadata about the app run.

2) A folder called 'Photos' which contains the sequence of images acquired.

The next cell will obliterate the middle directory (the directory just below the classification directory) and repackage each run's photos into a uniquely identifiable folder of its own (lacking the text file) at the level where the middle directories once sat.



In [10]:
# Find 'vestigial' directories to be deleted after unpacking.
presentVestDirs = np.array(glob(topDirName + '/Present/*'))
absentVestDirs = np.array(glob(topDirName + '/Absent/*'))

# Unpack the JPG photo files into identifiable directories.
allPresentSamples = np.array(glob(topDirName + '/Present/*/*'))
for sample in allPresentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    digestedTimestamp = re.split('_',digestedSample[3])
    newDir = digestedSample[2] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    
allAbsentSamples = np.array(glob(topDirName + '/Absent/*/*'))
for sample in allAbsentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    digestedTimestamp = re.split('_',digestedSample[3])
    newDir = digestedSample[2] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    
    
# Delete vestigial directories.
for direc in presentVestDirs:
    os.system('rm -rf ' + direc)
    
for direc in absentVestDirs:
    os.system('rm -rf ' + direc)

Look through all directories and find image sequences containing the file "IMG_9999.JPG". If found, sort and rename to an appropriate image sequence. This is necessary because when iOS reaches "IMG_9999.JPG" it wraps back around to "IMG_0000.JPG" for the next in the sequence. If this is left as is, the sorting operation that follows will sort the images into the incorrect timing order. 

In [11]:
# Get the filenames.
imgSeqsToCorrect = os.popen('cd ~/src/YPB-AI/; find -name "*IMG_9999.JPG"').read()
# Split into unique entities.
imgSeqsToCorrect = re.split('\n',imgSeqsToCorrect)
# Drop the last entry, it's empty.
imgSeqsToCorrect.pop();

# Change any files that don't start with 'IMG_9' (i.e., any files that come after the wraparound) to start with 'J'.
# This will mean that when they are sorted, they will come correctly after the wraparound.
for sample in imgSeqsToCorrect:
    digestedSample = re.split('/',sample)
    pathToDir = os.path.join(digestedSample[0],digestedSample[1],digestedSample[2],digestedSample[3])
    filesInDir = os.listdir(pathToDir)
    for i in range(len(filesInDir)):
        if 'IMG_9' not in filesInDir[i]:
            newFile = filesInDir[i].replace('I','J',1)
            cmdString = 'mv ' + os.path.join(pathToDir,filesInDir[i]) + ' ' + os.path.join(pathToDir,newFile)
            os.system(cmdString)

# Use a CNN to transform image sequences to a pickle file and organize the results.

In [3]:
import torch
import torchvision.models as models
from PIL import Image
import pickle

Use a pretrained CNN model (here using DenseNet121 because it has worked well in the past) with the final classifier layer chopped off.

In [4]:
# Import the CNN model and move it to CUDA if available.
model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
del model.classifier

if torch.cuda.is_available():
    model.to('cuda');

Define the function that will transform model output into a pickle file.

In [5]:
def transform_to_pickle(model,topdir):
    model.eval()
    ndx = 0
    topdirs = [direc for direc in os.listdir(topdir) if direc[0] is not '.']
    
    # Calculate the total number of stacks to transform.
    numStacks = 0
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        numStacks += len(directories)
    
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        
        
        # Do the transformation.
        for photoDir in directories:
            
            thisPhotoDir = os.path.join(thisClassDir, photoDir)
            ##pin pin fix##
            
#             print(thisPhotoDir)
#             print(os.path.isfile(photoDir))
#             continue
            
            if not os.path.isdir(thisPhotoDir):
                continue

            ## end ##
            files = [photofile for photofile in os.listdir(thisPhotoDir) if photofile.lower().endswith('jpg')]
            print("Transforming image stack {}".format(thisPhotoDir))
            files.sort()
            output = []
            # Make sure to appropriately transform the image size and normalize the RGB values, as the model expects. 
            for file in files:
                thisFile = os.path.join(thisPhotoDir,file)
                imgArr = np.float32(np.array(Image.open(thisFile).resize((224,224))).reshape([1,3,224,224]))
                imgArr[0][0] = ((imgArr[0][0] - 123.675)/58.395)
                imgArr[0][1] = ((imgArr[0][1] - 116.28)/57.12)
                imgArr[0][2] = ((imgArr[0][2] - 103.53)/57.375)
                if torch.cuda.is_available():
                    output.append(model(torch.cuda.FloatTensor(imgArr)))
                else:
                    output.append(model(torch.FloatTensor(imgArr)))

            FinalOutput = torch.FloatTensor(len(files),1,1024)
            for i in range(len(files)):
                FinalOutput[i] = output[i]
            pathStr = thisPhotoDir + '.pickle'
            with open(pathStr,'wb') as f:
                pickle.dump(FinalOutput,f)
                ndx += 1
                print("{}% complete.".format(ndx/numStacks*100))
    

Do the transformation

In [15]:
transform_to_pickle(model,topDirName)

Transforming image stack Bam1/Absent/6S_redBottle_V_Negative_30-Oct-2019_09-47-57
0.09049773755656108% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_H_Negative_30-Oct-2019_15-00-08
0.18099547511312217% complete.
Transforming image stack Bam1/Absent/6S_redBottle_H_Negative_29-Oct-2019_17-03-27
0.27149321266968324% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_H_Negative_30-Oct-2019_15-47-29
0.36199095022624433% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_V_Negative_30-Oct-2019_16-54-21
0.4524886877828055% complete.
Transforming image stack Bam1/Absent/6S_redBottle_V_Negative_30-Oct-2019_09-46-52
0.5429864253393665% complete.
Transforming image stack Bam1/Absent/6S_redBottle_V_Negative_30-Oct-2019_09-44-25
0.6334841628959276% complete.
Transforming image stack Bam1/Absent/6S_redBottle_V_Negative_30-Oct-2019_09-42-37
0.7239819004524887% complete.
Transforming image stack Bam1/Absent/6S_pillBottle_V_Negative_30-Oct-2019_18-34-02
0.814479

6.60633484162896% complete.
Transforming image stack Bam1/Absent/6S_pillBottle_V_Negative_30-Oct-2019_18-45-20
6.6968325791855206% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_V_Negative_30-Oct-2019_17-14-24
6.787330316742081% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_H_Negative_30-Oct-2019_15-27-00
6.877828054298643% complete.
Transforming image stack Bam1/Absent/6S_redBottle_V_Negative_30-Oct-2019_09-40-58
6.968325791855204% complete.
Transforming image stack Bam1/Absent/6S_redBottle_H_Negative_29-Oct-2019_16-53-02
7.0588235294117645% complete.
Transforming image stack Bam1/Absent/6S_redBottle_V_Negative_30-Oct-2019_10-05-11
7.149321266968325% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_H_Negative_30-Oct-2019_13-09-02
7.239819004524888% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_H_Negative_30-Oct-2019_12-35-30
7.330316742081449% complete.
Transforming image stack Bam1/Absent/6S_redBottle_V_Negative_30-Oct-2

13.21266968325792% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_V_Negative_30-Oct-2019_16-42-17
13.30316742081448% complete.
Transforming image stack Bam1/Absent/6S_pillBottle_V_Negative_30-Oct-2019_18-36-02
13.393665158371041% complete.
Transforming image stack Bam1/Absent/6S_redBottle_V_Negative_30-Oct-2019_09-54-06
13.484162895927602% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_H_Negative_30-Oct-2019_13-18-45
13.574660633484163% complete.
Transforming image stack Bam1/Absent/6S_redBottle_V_Negative_30-Oct-2019_11-57-41
13.665158371040723% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_V_Negative_30-Oct-2019_16-07-34
13.755656108597286% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_V_Negative_30-Oct-2019_16-06-17
13.846153846153847% complete.
Transforming image stack Bam1/Absent/6S_redBottle_H_Negative_29-Oct-2019_17-19-11
13.936651583710407% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_H_Negative

19.81900452488688% complete.
Transforming image stack Bam1/Absent/6S_redBottle_H_Negative_29-Oct-2019_16-58-12
19.90950226244344% complete.
Transforming image stack Bam1/Absent/6S_redBottle_V_Negative_30-Oct-2019_10-04-02
20.0% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_H_Negative_30-Oct-2019_12-14-44
20.09049773755656% complete.
Transforming image stack Bam1/Absent/6S_pillBottle_V_Negative_30-Oct-2019_18-35-02
20.18099547511312% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_V_Negative_30-Oct-2019_17-31-52
20.271493212669682% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_H_Negative_30-Oct-2019_13-16-47
20.361990950226243% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_V_Negative_30-Oct-2019_16-20-57
20.452488687782804% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_V_Negative_30-Oct-2019_17-20-10
20.542986425339365% complete.
Transforming image stack Bam1/Absent/6S_redBottle_V_Negative_30-Oct-2019_09-

26.42533936651584% complete.
Transforming image stack Bam1/Absent/6S_redBottle_H_Negative_29-Oct-2019_16-58-41
26.5158371040724% complete.
Transforming image stack Bam1/Absent/6S_redBottle_V_Negative_30-Oct-2019_11-58-01
26.60633484162896% complete.
Transforming image stack Bam1/Absent/6S_redBottle_V_Negative_30-Oct-2019_09-41-31
26.69683257918552% complete.
Transforming image stack Bam1/Absent/6S_pillBottle_V_Negative_30-Oct-2019_19-15-58
26.787330316742082% complete.
Transforming image stack Bam1/Absent/6S_pillBottle_V_Negative_30-Oct-2019_18-00-46
26.877828054298643% complete.
Transforming image stack Bam1/Absent/6S_redBottle_H_Negative_29-Oct-2019_16-59-47
26.968325791855204% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_H_Negative_30-Oct-2019_15-16-10
27.058823529411764% complete.
Transforming image stack Bam1/Absent/6S_pillBottle_V_Negative_30-Oct-2019_18-57-52
27.149321266968325% complete.
Transforming image stack Bam1/Absent/6S_filledBottle_V_Negative_30-Oct-20

...

67.87330316742081% complete.
Transforming image stack Bam1/Present/6S_filledBottle_V_30-Oct-2019_11-07-17
67.96380090497738% complete.
Transforming image stack Bam1/Present/6S_pillBottle_V_29-Oct-2019_14-08-31
68.05429864253394% complete.
Transforming image stack Bam1/Present/6S_filledBottle_H_29-Oct-2019_16-10-16
68.1447963800905% complete.
Transforming image stack Bam1/Present/6S_filledBottle_H_29-Oct-2019_15-36-59
68.23529411764706% complete.
Transforming image stack Bam1/Present/6S_pillBottle_H_29-Oct-2019_12-11-41
68.32579185520362% complete.
Transforming image stack Bam1/Present/6S_redBottle_V_30-Oct-2019_09-03-22
68.41628959276018% complete.
Transforming image stack Bam1/Present/6S_redBottle_V_30-Oct-2019_09-06-12
68.50678733031674% complete.
Transforming image stack Bam1/Present/6S_filledBottle_H_29-Oct-2019_16-18-22
68.5972850678733% complete.
Transforming image stack Bam1/Present/6S_redBottle_V_30-Oct-2019_09-27-15
68.68778280542986% complete.
Transforming image stack Bam1/Pr

75.02262443438914% complete.
Transforming image stack Bam1/Present/6S_filledBottle_H_29-Oct-2019_15-42-01
75.1131221719457% complete.
Transforming image stack Bam1/Present/6S_redBottle_V_30-Oct-2019_08-56-05
75.20361990950227% complete.
Transforming image stack Bam1/Present/6S_filledBottle_H_29-Oct-2019_16-06-49
75.29411764705883% complete.
Transforming image stack Bam1/Present/6S_pillBottle_V_29-Oct-2019_14-06-13
75.38461538461539% complete.
Transforming image stack Bam1/Present/6S_redBottle_V_30-Oct-2019_09-11-33
75.47511312217195% complete.
Transforming image stack Bam1/Present/6S_filledBottle_H_29-Oct-2019_15-54-23
75.56561085972851% complete.
Transforming image stack Bam1/Present/6S_redBottle_H_29-Oct-2019_10-08-59
75.65610859728507% complete.
Transforming image stack Bam1/Present/6S_redBottle_H_29-Oct-2019_10-08-11
75.74660633484163% complete.
Transforming image stack Bam1/Present/6S_redBottle_H_29-Oct-2019_10-48-37
75.83710407239819% complete.
Transforming image stack Bam1/Prese

82.17194570135746% complete.
Transforming image stack Bam1/Present/6S_filledBottle_H_29-Oct-2019_15-45-36
82.26244343891402% complete.
Transforming image stack Bam1/Present/6S_pillBottle_V_29-Oct-2019_14-23-14
82.35294117647058% complete.
Transforming image stack Bam1/Present/6S_filledBottle_V_30-Oct-2019_11-38-27
82.44343891402714% complete.
Transforming image stack Bam1/Present/6S_filledBottle_H_29-Oct-2019_16-00-46
82.5339366515837% complete.
Transforming image stack Bam1/Present/6S_filledBottle_H_29-Oct-2019_15-42-21
82.62443438914026% complete.
Transforming image stack Bam1/Present/6S_redBottle_H_29-Oct-2019_10-47-21
82.71493212669682% complete.
Transforming image stack Bam1/Present/6S_filledBottle_H_29-Oct-2019_16-06-18
82.80542986425338% complete.
Transforming image stack Bam1/Present/6S_pillBottle_V_29-Oct-2019_14-25-09
82.89592760180996% complete.
Transforming image stack Bam1/Present/6S_pillBottle_V_29-Oct-2019_14-00-30
82.98642533936652% complete.
Transforming image stack Ba

89.3212669683258% complete.
Transforming image stack Bam1/Present/6S_redBottle_V_30-Oct-2019_08-56-49
89.41176470588236% complete.
Transforming image stack Bam1/Present/6S_pillBottle_V_29-Oct-2019_14-01-56
89.50226244343892% complete.
Transforming image stack Bam1/Present/6S_pillBottle_H_29-Oct-2019_12-14-34
89.59276018099548% complete.
Transforming image stack Bam1/Present/6S_redBottle_H_29-Oct-2019_10-57-58
89.68325791855204% complete.
Transforming image stack Bam1/Present/6S_pillBottle_V_29-Oct-2019_14-27-43
89.7737556561086% complete.
Transforming image stack Bam1/Present/6S_pillBottle_H_29-Oct-2019_12-19-05
89.86425339366517% complete.
Transforming image stack Bam1/Present/6S_redBottle_V_30-Oct-2019_09-18-37
89.95475113122173% complete.
Transforming image stack Bam1/Present/6S_filledBottle_H_29-Oct-2019_16-20-28
90.04524886877829% complete.
Transforming image stack Bam1/Present/6S_pillBottle_H_29-Oct-2019_12-15-44
90.13574660633485% complete.
Transforming image stack Bam1/Present/

96.47058823529412% complete.
Transforming image stack Bam1/Present/6S_pillBottle_H_29-Oct-2019_12-06-22
96.56108597285068% complete.
Transforming image stack Bam1/Present/6S_pillBottle_H_29-Oct-2019_12-01-35
96.65158371040724% complete.
Transforming image stack Bam1/Present/6S_filledBottle_H_29-Oct-2019_16-03-15
96.7420814479638% complete.
Transforming image stack Bam1/Present/6S_pillBottle_V_29-Oct-2019_14-17-50
96.83257918552036% complete.
Transforming image stack Bam1/Present/6S_redBottle_V_30-Oct-2019_09-14-24
96.92307692307692% complete.
Transforming image stack Bam1/Present/6S_redBottle_H_29-Oct-2019_10-17-30
97.01357466063348% complete.
Transforming image stack Bam1/Present/6S_filledBottle_V_30-Oct-2019_11-33-17
97.10407239819004% complete.
Transforming image stack Bam1/Present/6S_pillBottle_H_29-Oct-2019_12-02-56
97.1945701357466% complete.
Transforming image stack Bam1/Present/6S_filledBottle_V_30-Oct-2019_11-22-13
97.28506787330316% complete.
Transforming image stack Bam1/Pre

Clean up the directory, removing everything but the pickle files.

In [16]:
# Assemble all the pickle files and remove the rest.
classDirs = [direc for direc in os.listdir(topDirName) if direc[0] is not '.']
for classDir in classDirs:
    newDir = os.path.join(topDirName,classDir)
    fileDirs = [direc for direc in os.listdir(newDir) if direc[0] is not '.' and not direc.lower().endswith('.pickle')]
    for fileDir in fileDirs:
        pickleFileDir = os.path.join(newDir,fileDir)
        os.system('rm -rf ' + pickleFileDir)

# Split the directory into train/test/validation sets

In [7]:
import random

In [6]:
# finalDirName = 'Split_Data'
finalDirName = 'Split_Data_Bam1_no_pillBottle'

In [8]:
def split_folders(inputDir, outputDir, trainValTestRatio, seed):
    # Assume inputDir is organized as 'inputDir/class/topFolder/photos'
    classes = os.listdir(inputDir)
    # Exclude folders where '.' is the first character.
    classes = [directory for directory in classes if directory[0] != '.']
    numClasses = len(classes)
    # Create output directories
    os.system('mkdir ' + outputDir)
    os.system('mkdir ' + outputDir + '/train')
    os.system('mkdir ' + outputDir + '/valid')
    os.system('mkdir ' + outputDir + '/test')
    # Pull files and put into the appropriate folders.
    for directory in classes:
        files = os.listdir(os.path.join(inputDir, directory))
        files.sort()
        random.seed(seed)
        random.shuffle(files)
        numFiles = len(files)
        # Get the indices.
        ndxAmts = np.intc((np.round(trainValTestRatio / np.sum(trainValTestRatio) * numFiles)))
        numNdxes = np.sum(ndxAmts)
        if numNdxes != numFiles:
            ndxAmts[0] += numFiles-numNdxes
        trainFiles = files[0:ndxAmts[0]]
        valFiles = files[ndxAmts[0]:ndxAmts[0]+ndxAmts[1]]
        testFiles = files[ndxAmts[0]+ndxAmts[1]:]
        # Put the training directories in the appropriate folder
        # First make the class directories.
        os.system('mkdir ' + outputDir + '/train/' + directory)
        os.system('mkdir ' + outputDir + '/valid/' + directory)
        os.system('mkdir ' + outputDir + '/test/' + directory)
        for file in trainFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/train/' + directory + '/' + file)
        for file in valFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/valid/' + directory + '/' + file)
        for file in testFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/test/' + directory + '/' + file)

In [9]:
split_folders(topDirName, finalDirName,[0.8,0.1,0.1],7778)